# Tabular Playground Series Oct 2021

In [1]:
# Installing the modules

!pip3 install kaggle

In [17]:
# Required modules

import numpy as np
import pandas as pd

from zipfile import ZipFile
from google.colab import files
from matplotlib import pyplot as plt

from xgboost import XGBClassifier

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Config

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)

In [3]:
# Upload the creds

files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"anurag1817","key":"a91861c07392ba9138e2e8b5e23e168a"}'}

In [4]:
# Create kaggle folder

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
# Test the command

!kaggle competitions download -c tabular-playground-series-oct-2021

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
# Extract the zip file

with ZipFile('/content/train.csv.zip', 'r') as zf:
    zf.extractall('./')

with ZipFile('/content/test.csv.zip', 'r') as zf:
    zf.extractall('./')

with ZipFile('/content/sample_submission.csv.zip', 'r') as zf:
    zf.extractall('./')

In [6]:
# Load the train data

train = pd.read_csv('train.csv', index_col='id')
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.147295,0.026177,0.106613,0.200924,0.713191,0.155750,0.557335,0.341702,0.285720,0.230396,0.203957,0.509588,0.706972,1,0.007793,0.247765,0.263750,0.259555,0.231730,0.138379,0.197824,0.054392,0.194153,0.281500,0.034818,0.025334,0.114432,0.139203,0.246157,0.251371,0.701423,...,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1
1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,0.493411,0.107277,0.231828,0.457150,0.395200,0.617088,0.459358,0.209225,0.201098,0.199383,0.366578,0.585788,1,0.285311,0.400367,0.162493,0.249365,0.141160,0.133688,0.247906,0.139251,0.216444,0.109674,0.033018,0.017458,0.189336,0.168785,0.184251,0.202753,0.218451,...,1,0,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1
2,0.182583,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,0.286813,0.139532,0.107222,0.247791,0.631949,0.347463,0.642173,0.257763,0.162548,0.327377,0.193583,0.495440,0.636742,0,0.007133,0.309747,0.221081,0.284810,0.230828,0.138271,0.199742,0.060408,0.146746,0.208131,0.035977,0.022631,0.113542,0.274871,0.182770,0.151502,0.570035,...,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1
3,0.180240,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,0.260886,0.147122,0.105433,0.287755,0.455777,0.247971,0.616628,0.335907,0.337025,0.239127,0.176163,0.538269,0.706468,1,0.008829,0.353799,0.219977,0.266858,0.145617,0.138590,0.234925,0.059817,0.140886,0.205023,0.319312,0.009972,0.112291,0.288915,0.332632,0.140831,0.473845,...,1,1,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
4,0.177172,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,0.158321,0.260239,0.102561,0.265285,0.503776,0.269776,0.545945,0.319548,0.278538,0.214922,0.200239,0.534551,0.728652,0,0.004840,0.323546,0.166292,0.285516,0.208651,0.200394,0.198155,0.414729,0.251997,0.193405,0.034490,0.016465,0.197312,0.207429,0.255857,0.139875,0.321039,...,0,0,0,1,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1


In [7]:
# Inspecting the data

train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Columns: 286 entries, f0 to target
dtypes: float64(240), int64(46)
memory usage: 2.1 GB


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.00000,1000000.000000
mean,0.214334,0.460218,0.129253,0.277598,0.580710,0.416619,0.386532,0.654858,0.462256,0.258031,0.181128,0.131991,0.278548,0.595653,0.292308,0.593562,0.336334,0.243227,0.225491,0.221857,0.481103,0.700318,0.548061,0.063248,0.284030,0.234265,0.260321,0.196285,0.181265,0.228721,0.118508,0.236993,0.226346,0.080244,0.052288,0.131734,0.205375,0.218599,0.257150,0.485651,...,0.804147,0.679138,0.279384,0.408040,0.434857,0.491085,0.281427,0.830737,0.342218,0.590451,0.278183,0.187018,0.539352,0.390343,0.266291,0.347034,0.173990,0.442588,0.495109,0.407147,0.262252,0.393640,0.302201,0.203648,0.469065,0.426087,0.256545,0.310854,0.345492,0.244899,0.250096,0.137164,0.144793,0.130667,0.139210,0.199331,0.156065,0.183741,0.15468,0.500485
std,0.053320,0.101316,0.120805,0.063163,0.115338,0.058231,0.133457,0.065158,0.129439,0.119081,0.157788,0.077778,0.071025,0.075483,0.090123,0.086750,0.085286,0.068466,0.044640,0.044283,0.050815,0.075411,0.497685,0.118831,0.094791,0.073762,0.047038,0.081383,0.074089,0.049318,0.099597,0.080056,0.106163,0.109916,0.111535,0.057181,0.081033,0.057003,0.095631,0.107811,...,0.396856,0.466808,0.448697,0.491471,0.495738,0.499921,0.449696,0.374984,0.474452,0.491751,0.448104,0.389926,0.498449,0.487827,0.442018,0.476027,0.379101,0.496693,0.499976,0.491303,0.439859,0.488557,0.459212,0.402710,0.499042,0.494507,0.436726,0.462843,0.475529,0.430028,0.433068,0.344021,0.351892,0.337036,0.346166,0.399498,0.362917,0.387273,0.36160,0.500000
min,0.041790,0.022016,0.000381,0.000000,0.000000,0.000959,0.000592,0.017994,0.000990,0.051183,0.000000,0.000000,0.037846,0.024079,0.003086,0.011323,0.000000,0.000818,0.014163,0.010135,0.081429,0.067804,0.000000,0.000182,0.016970,0.023514,0.047751,0.013686,0.009447,0.030857,0.000000,0.057145,0.019329,0.001174,0.000216,0.000000,0.068346,0.031468,0.033686,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.181676,0.389215,0.017692,0.235342,0.497938,0.374390,0.317815,0.615372,0.363753,0.164559,0.027519,0.103078,0.236119,0.565547,0.239312,0.544008,0.304199,0.204140,0.200744,0.197627,0.450656,0.660346,0.000000,0.006746,0.221265,0.184328,0.231221,0.141

In [11]:
# Correlation matrix

train.corr().unstack().sort_values(kind="quicksort").tolist()

[-0.5140738672316877,
 -0.5140738672316877,
 -0.15451716690683484,
 -0.15451716690683484,
 -0.08075877179752822,
 -0.08075877179752822,
 -0.0680800990004023,
 -0.0680800990004023,
 -0.06569354912269235,
 -0.06569354912269235,
 -0.05147411612639111,
 -0.05147411612639111,
 -0.04355693954703645,
 -0.04355693954703645,
 -0.038657872321664614,
 -0.038657872321664614,
 -0.037850202910401275,
 -0.037850202910401275,
 -0.03713186697246113,
 -0.03713186697246113,
 -0.034787999285472805,
 -0.034787999285472805,
 -0.031420249892511534,
 -0.031420249892511534,
 -0.03099035587029426,
 -0.03099035587029426,
 -0.03070100716005573,
 -0.03070100716005573,
 -0.030397248582290967,
 -0.030397248582290967,
 -0.03033513109363579,
 -0.03033513109363579,
 -0.029953282963212598,
 -0.029953282963212598,
 -0.02970188829319407,
 -0.02970188829319407,
 -0.02937924115461083,
 -0.02937924115461083,
 -0.02932388204182704,
 -0.02932388204182704,
 -0.028041983639436736,
 -0.028041983639436736,
 -0.027768492855713475,


In [8]:
# Load the test data

test = pd.read_csv('test.csv', index_col='id')
test.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000000,0.178216,0.435617,0.010230,0.202074,0.390170,0.324221,0.221722,0.738894,0.582588,0.343770,0.028081,0.102890,0.263804,0.607342,0.172180,0.491036,0.326189,0.252032,0.222362,0.242405,0.471047,0.723153,0,0.151190,0.226410,0.305478,0.269863,0.139583,0.141646,0.200469,0.059474,0.283620,0.244343,0.033877,0.010561,0.112109,0.193434,0.182685,0.188303,0.419009,...,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,1,1,1,0
1000001,0.181250,0.476455,0.022413,0.283146,0.598020,0.349508,0.283467,0.721575,0.268990,0.208373,0.278440,0.107586,0.402494,0.594466,0.193365,0.592103,0.345882,0.257781,0.201138,0.215895,0.449694,0.627330,1,0.133139,0.438526,0.215876,0.242926,0.156744,0.143899,0.697532,0.210877,0.268969,0.209255,0.040739,0.014843,0.113464,0.187680,0.206614,0.389319,0.535426,...,1,1,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1000002,0.159721,0.451202,0.259649,0.365274,0.594634,0.413502,0.249318,0.642339,0.411104,0.246891,0.029237,0.105428,0.230432,0.622365,0.190519,0.573442,0.531088,0.169617,0.224070,0.204311,0.453146,0.723547,0,0.008878,0.457426,0.199645,0.301306,0.217489,0.139284,0.200483,0.057339,0.488494,0.116244,0.040174,0.015913,0.113177,0.137054,0.182741,0.250186,0.572463,...,1,1,1,0,1,0,1,0,1,0,1,1,0,1,0,0,0,0,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0
1000003,0.182424,0.520976,0.095344,0.327742,0.741830,0.358711,0.270077,0.601662,0.297742,0.252829,0.550020,0.225594,0.199320,0.667981,0.298132,0.598126,0.332168,0.240837,0.209166,0.210716,0.529991,0.755527,0,0.007472,0.340279,0.181562,0.449528,0.178485,0.144564,0.198985,0.057868,0.135401,0.525434,0.312709,0.011679,0.201902,0.198581,0.234629,0.329007,0.539557,...,0,1,1,0,1,0,1,0,1,0,0,1,1,1,1,0,0,0,0,1,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0
1000004,0.229329,0.336513,0.023511,0.300913,0.668738,0.481586,0.545660,0.667849,0.546045,0.202731,0.272089,0.107391,0.342519,0.634580,0.299089,0.622012,0.427928,0.275467,0.230340,0.192903,0.442950,0.650836,1,0.008275,0.296159,0.165649,0.251093,0.145741,0.140904,0.239641,0.055761,0.358701,0.118698,0.037337,0.013743,0.229668,0.135660,0.244730,0.243071,0.417749,...,1,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0


In [9]:
# Missing values

if train.isna().any().any():
    print(train.isna().sum()/train.shape[0])
else:
    print("No Missing values")

No Missing values


In [10]:
# Seperating out features and labels

X = train.drop(['target'], axis=1)
y = train['target']

In [11]:
# Train Test Split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, random_state=88)

## PassiveAggressiveClassifier

In [18]:
# Model Definition

model = RandomForestClassifier(verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 31.2min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

In [19]:
# Model Prediction

test_pred = model.predict(test)
test_pred

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   14.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   30.6s finished


array([1, 0, 1, ..., 0, 1, 0])

In [20]:
# Generating output file

submission = pd.read_csv('sample_submission.csv')
submission['target'] = test_pred
submission.to_csv('output.csv', index=False)

In [21]:
# Submission

!kaggle competitions submit -c tabular-playground-series-oct-2021 -f output.csv -m "RF"

100% 4.77M/4.77M [00:02<00:00, 1.83MB/s]
Successfully submitted to Tabular Playground Series - Oct 2021